## A  Simple Cast Study to Understand about Robust Optimization in Process Optimzation

In [59]:
from pyomo.environ import *
import os

### Problem Description

Imagine a process with two reactions when written in Mass Balance:

$$ A \longrightarrow 0.4 C + 0.6 D \quad \quad \text{RXN1}$$
$$ B \longrightarrow 0.7 C + 0.3 D \quad \quad \text{RXN2}$$

The two reactions happen together in the reactor, and the product stream goes to a separator that performs a complete binary separation.

![Optional Alt Text](illustrations/toy.png)

Nomenclature

Sets

\begin{align*}
    & I && \quad \text{Set of Streams: \{1,2,3,4,5,6\}} \\
    & J && \quad \text{Set of Species: \{A,B,C,D\}} \\
    & T && \quad \text{Set of Technologies: \{R1, R2, SEP1\}} \\
\end{align*}

Parameters

\begin{align*}
    & Q^{Load}_{j} && \quad \text{Mass flow rate of stream i for species j} \\
    & \theta_{t,j} \quad  t \in \{R1, R2\} && \quad \text{Conversion of technology t for species j} \\
    & \alpha_{t} && \quad \text{Cost coefficient of technology t} \\
    & \beta_{j} && \quad \text{Price of species j}
\end{align*}

Variables

\begin{align*}
    & F_{i,j} \quad \text{Mass flow rate of stream i for species j} \\
    & W_{t} \quad \text{Capacity of technology t} \\
    & A_{t} \quad \text{Cost of technology t} \\
    & B_{j} \quad \text{Cost of species, Negative for products}
\end{align*}


### Feed & Demand Constraints

First we begin with total supply of each feedstock we have.

\begin{align*}
    & F_{1, A} \le Q^{Sup}_{A} \\
    & F_{2, B} \le Q^{Sup}_{B} \\
\end{align*}

\begin{align*}
    & F_{5, C} \ge Q^{Dem}_{C} \\
    & F_{6, D} \ge Q^{Dem}_{D} \\
\end{align*}

We also requires the feed to be pure streams:

\begin{align*}
    & F_{1, j} = 0 \text{   if } j \neq A \\
    & F_{2, j} = 0 \text{   if } j \neq B \\
\end{align*}


### Mass Balances

For the reactions, we have

\begin{align*}
    & F_{3, j} - F_{1,j} = \theta_{R1,j} F_{1,j} \\
\end{align*}

\begin{align*}
    & F_{4, j} - F_{2,j} = \theta_{R2,j} F_{2,j} \\
\end{align*}

For the Separation:

\begin{align*}
    & F_{3, j} + F_{4,j} = F_{5,j} \text{   if } j \neq D \\
\end{align*}

\begin{align*}
    & F_{3, j} + F_{4,j} = F_{6,j} \text{   if } j = D \\
\end{align*}

\begin{align*}
    & F_{3, j} + F_{4,j} = F_{5,j} + F_{6,j}  \\
\end{align*}

This completes our mass balance.

### Design Feasibility

For the capacities:

\begin{align*}
    & F_{1, A} \le W_{R1} \\
    & F_{2, B} \le W_{R2} \\
    & \sum_{j} (F_{3, j} + F_{4, j}) \le W_{R3} \\
\end{align*}

This completes the design feasibility  constraints if we only consider the capacity.

### Economics

In this case, we assume a linear relationship between both capital and operating cost associated with technologies.

\begin{align*}
    & A_{t} = \alpha_{t} W_{t}  \\
\end{align*}

As for the material cost/sales:

\begin{align*}
    & B_{A} = \beta_{A} F_{1,A}  \\
    & B_{B} = \beta_{B} F_{2,B}  \\
    & B_{C} = \beta_{C} F_{5,C}  \\
    & B_{D} = \beta_{D} F_{6,D}  \\
\end{align*}


### Deterministic Model


$$  Min_{\{F_{1,A}, F_{2,B}\}} \sum_{t \in T} W_{t} + \sum_{j \in J} P_{j}  $$

subject to

\begin{align*}
    & F_{1, A} \le Q^{Load}_{A} \\
    & F_{2, B} \le Q^{Load}_{B} \\
    & F_{5, C} \ge Q^{Dem}_{C} \\
    & F_{6, D} \ge Q^{Dem}_{D} \\
    & F_{1, j} = 0 \text{   if } j \neq A \\
    & F_{2, j} = 0 \text{   if } j \neq B \\
    & F_{3, j} - F_{1,j} = \theta_{R1,j} F_{1,j} \\
    & F_{4, j} - F_{2,j} = \theta_{R2,j} F_{2,j} \\
    & F_{3, j} + F_{4,j} = F_{5,j} \text{   if } j \neq D \\
    & F_{3, j} + F_{4,j} = F_{6,j} \text{   if } j = D \\
    & F_{3, j} + F_{4,j} = F_{5,j} + F_{6,j}  \\
    & F_{1, A} \le W_{R1} \\
    & F_{2, B} \le W_{R2} \\
    & \sum_{j} (F_{3, j} + F_{4, j}) \le W_{R3} \\
    & W_{R1} = \alpha_{R1} F_{1,A}  \\
    & W_{R2} = \alpha_{R2} F_{2,B}  \\
    & W_{SEP1} = \alpha_{R3} \sum_{j}(F_{3,j} + F_{4,j}) \\
    & P_{A} = \beta_{A} F_{1,A}  \\
    & P_{B} = \beta_{B} F_{2,B}  \\
    & P_{C} = \beta_{C} F_{5,C}  \\
    & P_{D} = \beta_{D} F_{6,D}  \\
\end{align*}


In [60]:
def build_DM():
    m = ConcreteModel()

    m.i = Set(initialize=[1,2,3,4,5,6], doc="streams")
    m.j = Set(initialize=["A","B","C","D"])
    m.t = Set(initialize=["R1","R2","SEP1"])

    m.Qsup = Param(m.j,initialize={"A": 100, "B": 100})
    m.Qdem = Param(m.j,initialize={"C": 50, "D": 50})
    m.theta = Param(m.t,m.j,initialize={("R1","A"): -1, ("R1","B"): 0, ("R1","C"): 0.4, ("R1","D"): 0.6,
                                        ("R2","A"): 0, ("R2","B"): -1, ("R2","C"): 0.7, ("R2","D"): 0.3,
                                        })
    m.alpha = Param(m.t,initialize={"R1": 1.5, "R2": 1, "SEP1": 2})
    m.beta = Param(m.j,initialize={"A": 5, "B": 6, "C": -7, "D": -8})

    m.F = Var(m.i, m.j, domain=NonNegativeReals, doc="Flow rate of stream i speices j")
    m.W = Var(m.t, domain=NonNegativeReals, doc="Technology Capacity")
    m.A = Var(m.t, domain=NonNegativeReals, doc="Technology Cost")
    m.B = Var(m.j, domain=Reals, doc="Material Cost")

    def _csupply(m, j):
        if not j in {"A","B"}:
            return Constraint.Skip
        if j == "A":
            return m.F[1,j] <= m.Qsup[j]
        elif j == "B":
            return m.F[2,j] <= m.Qsup[j]
        raise IndexError
    m.csupply = Constraint(m.j, rule=_csupply)

    def _cdemand(m, j):
        if not j in {"C","D"}:
            return Constraint.Skip
        if j == "C":
            return m.F[5,j] >= m.Qdem[j]
        elif j == "D":
            return m.F[6,j] >= m.Qdem[j]
        raise IndexError
    m.cdemand = Constraint(m.j, rule=_cdemand)

    def _cfeed(m, i, j):
        if not i in {1,2}:
            return Constraint.Skip
        if not (i,j) in {(1,"A"), (2,"B")}:
            return m.F[i,j] == 0
        return Constraint.Skip
    m.cfeed = Constraint(m.i, m.j, rule=_cfeed)

    def _cR1(m, j):
        return m.F[3,j] - m.F[1,j] == m.theta["R1",j] * m.F[1,"A"]
    m.cR1 = Constraint(m.j, rule=_cR1)

    def _cR2(m, j):
        return m.F[4,j] - m.F[2,j] == m.theta["R2",j] * m.F[2,"B"]
    m.cR2 = Constraint(m.j, rule=_cR2)

    def _csep(m, j):
        if j != "D":
            return m.F[3,j] + m.F[4,j] == m.F[5,j]
        return m.F[3,j] + m.F[4,j] ==  m.F[6,j]
    m.csep = Constraint(m.j, rule=_csep)

    def _csep2(m, j):
        return m.F[3,j] + m.F[4,j] == m.F[5,j] + m.F[6,j]
    m.csep2 = Constraint(m.j, rule=_csep2)

    def _ccap(m, t):
        if t == "R1":
            return m.W[t] >= m.F[1,"A"]
        elif t == "R2":
            return m.W[t] >= m.F[2,"B"]
        elif t == "SEP1":
            return m.W[t] >= sum(m.F[3,j] + m.F[4,j] for j in m.j)
        raise IndexError
    m.ccap = Constraint(m.t, rule=_ccap)

    def _techcost(m, t):
        return m.alpha[t] * m.W[t] == m.A[t]
    m.techcost = Constraint(m.t, rule=_techcost)

    def _matcost(m, j):
        if j == "A":
            return m.beta[j] * m.F[1, j] == m.B[j]
        elif j == "B":
            return m.beta[j] * m.F[2, j] == m.B[j]
        elif j == "C":
            return m.beta[j] * m.F[5, j] == m.B[j]
        elif j == "D":
            return m.beta[j] * m.F[6, j] == m.B[j]
        raise IndexError
    m.matcost = Constraint(m.j, rule=_matcost)

    m.obj = Objective(expr=sum(m.A[t] for t in m.t) + sum(m.B[j] for j in m.j),  sense=minimize)

    return m

m1 = build_DM()

In [61]:
def solve_with_gams(m, solver="cplex"):
    opt1 = SolverFactory('gams')
    io_options = dict()

    io_options['solver'] = solver
    res1 = opt1.solve(m,
        tee=True,
        keepfiles=True,
        add_options = ['option reslim=1000; option optcr=0.0;  option limrow=5000'],
        tmpdir=os.getcwd()+"/output",
        io_options=io_options)
    return res1

def solve_with_gurobi(m):
    opt1 = SolverFactory("gurobi_persistent")
    opt1.set_instance(m)
    res1 = opt1.solve()
    return res1

In [62]:
solve_with_gams(m1)
# m1.write("m1.lp", io_options={'symbolic_solver_labels': True})

--- Job model.gms Start 12/03/23 01:08:20 43.2.0 859d62d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\43\gmsprmNT.txt
    C:\Users\justi\OneDrive\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input D:\Optimization-Notebook\RO\output\model.gms
    Output D:\Optimization-Notebook\RO\output\output.lst
    ScrDir D:\Optimization-Notebook\RO\output\225a\
    SysDir C:\GAMS\43\
    CurDir D:\Optimization-Notebook\RO\output\
    LogOption 3
Licensee: Small MUD - 5 User License                     G221121|0002AP-GEN
          University of Delaware, Chemical and Biomolecular EngineeriDC3967
          C:\GAMS\43\gamslice.txt
          License Admin: Marianthi Ierapetritou, mgi@udel.edu              
Processor information: 1 socket(s), 10 core(s), and 20 thread(s) available
GAMS 43.2.0   Copyright (C) 1987-2023 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(260) 2 Mb
--- Starting execution: elapsed 0:00:00.004
--- model.gms(131) 3 Mb
--- 

{'Problem': [{'Name': 'D:\\Optimization-Notebook\\RO/output\\model.gms', 'Lower bound': nan, 'Upper bound': 114.28571428571433, 'Number of objectives': 1, 'Number of constraints': 37.0, 'Number of variables': 35.0, 'Number of binary variables': None, 'Number of integer variables': 0.0, 'Number of continuous variables': 35.0, 'Number of nonzeros': 91.0, 'Sense': 'minimize'}], 'Solver': [{'Name': 'GAMS (43, 2, 0, 0)', 'Status': 'ok', 'Return code': 0, 'Message': None, 'User time': 0.008000107482076, 'System time': None, 'Wallclock time': None, 'Termination condition': 'optimal', 'Termination message': None}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [63]:
m1.A.pprint(), m1.B.pprint()

A : Technology Cost
    Size=3, Index=t
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
      R1 :     0 :              150.0 :  None : False : False : NonNegativeReals
      R2 :     0 : 14.285714285714286 :  None : False : False : NonNegativeReals
    SEP1 :     0 : 228.57142857142858 :  None : False : False : NonNegativeReals
B : Material Cost
    Size=4, Index=j
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      A :  None :              500.0 :  None : False : False :  Reals
      B :  None :  85.71428571428572 :  None : False : False :  Reals
      C :  None :             -350.0 :  None : False : False :  Reals
      D :  None : -514.2857142857143 :  None : False : False :  Reals


(None, None)

In [64]:
m1.F[1,"A"].pprint(), m1.F[2,"B"].pprint(), m1.F[5,"C"].pprint(), m1.F[6,"D"].pprint()

{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 'A') :     0 : 100.0 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value              : Upper : Fixed : Stale : Domain
    (2, 'B') :     0 : 14.285714285714286 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
    (5, 'C') :     0 :  50.0 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value             : Upper : Fixed : Stale : Domain
    (6, 'D') :     0 : 64.28571428571429 :  None : False : False : NonNegativeReals


(None, None, None, None)

In [65]:
# for constraint in m1.component_objects(Constraint, active=True):
#     constraint_object = getattr(m1, str(constraint))
#     for index in constraint_object:
#         print(f"Constraint {constraint}: {constraint_object[index].expr}")

As we can see, in the nominal condition, the system is trying select rxn A  while doing minimum rxn B for demand constraints. Now, we do an experiment to formulate a robust version that considers a 20% uncertainty in Feed A and B.

### Robust Formulation

We first try to incorporate the constraint only at the variable directly associated with the uncertainty parameters.

\begin{align*}
    & F_{1, A} + max\{-\epsilon_A Q^{Sup}_{A} \} \le Q^{Sup}_{A}  \\
    & F_{2, B} + max\{-\epsilon_B Q^{Sup}_{B} \} \le Q^{Sup}_{B} \\
    & -0.3 \le \epsilon_A \le 0.1 \\
    & -0.1 \le \epsilon_B \le 0.2 \\
\end{align*}

Given that it is a box uncertainty set and the subproblem is linear with nonnegative variables, we can apply properties. However, to preserve the problem in more general way, we do below so it applies to subproblems that we cannot apply properties to.

\begin{align*}
    & F_{1, A} + u_A  \le Q^{Sup}_{A} \\
    & F_{2, B} + u_B  \le Q^{Sup}_{B} \\
    & -\epsilon_A Q^{Sup}_{A} \le u_A \\
    & -\epsilon_B Q^{Sup}_{B} \le u_B \\
    & -0.2 \le \epsilon_A \le 0.2 \\
    & -0.1 \le \epsilon_B \le 0.1 \\
\end{align*}

We leave everything else intact

In [66]:
def build_DM():
    m = ConcreteModel()

    m.i = Set(initialize=[1,2,3,4,5,6], doc="streams")
    m.j = Set(initialize=["A","B","C","D"])
    m.t = Set(initialize=["R1","R2","SEP1"])
    m.jfeed = Set(initialize=["A", "B"])

    m.Qsup = Param(m.j,initialize={"A": 100, "B": 100})
    m.Qdem = Param(m.j,initialize={"C": 50, "D": 50})
    m.theta = Param(m.t,m.j,initialize={("R1","A"): -1, ("R1","B"): 0, ("R1","C"): 0.4, ("R1","D"): 0.6,
                                        ("R2","A"): 0, ("R2","B"): -1, ("R2","C"): 0.7, ("R2","D"): 0.3,
                                        })
    m.alpha = Param(m.t,initialize={"R1": 1.5, "R2": 1, "SEP1": 2})
    m.beta = Param(m.j,initialize={"A": 5, "B": 6, "C": -7, "D": -8})

    m.F = Var(m.i, m.j, domain=NonNegativeReals, doc="Flow rate of stream i speices j")

    m.eps = Var(m.jfeed, domain=Reals, doc="uncertainty(Phi: adjustable parameter)")
    m.u = Var(m.jfeed, domain=Reals, doc="aux var: maximum of uncertainty")

    for jf in m.jfeed:
        if jf == "A":
            m.eps[jf].setlb(-0.2)
            m.eps[jf].setub(0.2)
        elif jf == "B":
            m.eps[jf].setlb(-0.1)
            m.eps[jf].setub(0.1)
        else:
            pass

    m.W = Var(m.t, domain=NonNegativeReals, doc="Technology Capacity")
    m.A = Var(m.t, domain=NonNegativeReals, doc="Technology Cost")
    m.B = Var(m.j, domain=Reals, doc="Material Cost")

    def _csupply(m, j):
        if j == "A":
            return m.F[1,j] + m.u[j] <= m.Qsup[j]
        elif j == "B":
            return m.F[2,j] + m.u[j] <= m.Qsup[j]
        raise IndexError
    m.csupply = Constraint(m.jfeed, rule=_csupply)

    def _csupply_ro(m, j):
        if j == "A":
            return -m.Qsup[j] * m.eps[j] <= m.u[j]
        elif j == "B":
            return -m.Qsup[j] * m.eps[j] <= m.u[j]
        raise IndexError
    m.csupply_ro = Constraint(m.jfeed, rule=_csupply_ro)

    def _cdemand(m, j):
        if not j in {"C","D"}:
            return Constraint.Skip
        if j == "C":
            return m.F[5,j] >= m.Qdem[j]
        elif j == "D":
            return m.F[6,j] >= m.Qdem[j]
        raise IndexError
    m.cdemand = Constraint(m.j, rule=_cdemand)

    def _cfeed(m, i, j):
        if not i in {1,2}:
            return Constraint.Skip
        if not (i,j) in {(1,"A"), (2,"B")}:
            return m.F[i,j] == 0
        return Constraint.Skip
    m.cfeed = Constraint(m.i, m.j, rule=_cfeed)

    def _cR1(m, j):
        return m.F[3,j] - m.F[1,j] == m.theta["R1",j] * m.F[1,"A"]
    m.cR1 = Constraint(m.j, rule=_cR1)

    def _cR2(m, j):
        return m.F[4,j] - m.F[2,j] == m.theta["R2",j] * m.F[2,"B"]
    m.cR2 = Constraint(m.j, rule=_cR2)

    def _csep(m, j):
        if j != "D":
            return m.F[3,j] + m.F[4,j] == m.F[5,j]
        return m.F[3,j] + m.F[4,j] ==  m.F[6,j]
    m.csep = Constraint(m.j, rule=_csep)

    def _csep2(m, j):
        return m.F[3,j] + m.F[4,j] == m.F[5,j] + m.F[6,j]
    m.csep2 = Constraint(m.j, rule=_csep2)

    def _ccap(m, t):
        if t == "R1":
            return m.W[t] >= m.F[1,"A"]
        elif t == "R2":
            return m.W[t] >= m.F[2,"B"]
        elif t == "SEP1":
            return m.W[t] >= sum(m.F[3,j] + m.F[4,j] for j in m.j)
        raise IndexError
    m.ccap = Constraint(m.t, rule=_ccap)

    def _techcost(m, t):
        return m.alpha[t] * m.W[t] == m.A[t]
    m.techcost = Constraint(m.t, rule=_techcost)

    def _matcost(m, j):
        if j == "A":
            return m.beta[j] * m.F[1, j] == m.B[j]
        elif j == "B":
            return m.beta[j] * m.F[2, j] == m.B[j]
        elif j == "C":
            return m.beta[j] * m.F[5, j] == m.B[j]
        elif j == "D":
            return m.beta[j] * m.F[6, j] == m.B[j]
        raise IndexError
    m.matcost = Constraint(m.j, rule=_matcost)

    m.obj = Objective(expr=sum(m.A[t] for t in m.t) + sum(m.B[j] for j in m.j),  sense=minimize)

    return m

m2 = build_DM()

In [67]:
solve_with_gams(m2, solver="ipopth")

--- Job model.gms Start 12/03/23 01:08:21 43.2.0 859d62d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\43\gmsprmNT.txt
    C:\Users\justi\OneDrive\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input D:\Optimization-Notebook\RO\output\model.gms
    Output D:\Optimization-Notebook\RO\output\output.lst
    ScrDir D:\Optimization-Notebook\RO\output\225a\
    SysDir C:\GAMS\43\
    CurDir D:\Optimization-Notebook\RO\output\
    LogOption 3
Licensee: Small MUD - 5 User License                     G221121|0002AP-GEN
          University of Delaware, Chemical and Biomolecular EngineeriDC3967
          C:\GAMS\43\gamslice.txt
          License Admin: Marianthi Ierapetritou, mgi@udel.edu              
Processor information: 1 socket(s), 10 core(s), and 20 thread(s) available
GAMS 43.2.0   Copyright (C) 1987-2023 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(278) 2 Mb
--- Starting execution: elapsed 0:00:00.008
--- model.gms(143) 3 Mb
--- 

{'Problem': [{'Name': 'D:\\Optimization-Notebook\\RO/output\\model.gms', 'Lower bound': nan, 'Upper bound': 112.49999999646741, 'Number of objectives': 1, 'Number of constraints': 39.0, 'Number of variables': 39.0, 'Number of binary variables': None, 'Number of integer variables': 0.0, 'Number of continuous variables': 39.0, 'Number of nonzeros': 97.0, 'Sense': 'minimize'}], 'Solver': [{'Name': 'GAMS (43, 2, 0, 0)', 'Status': 'ok', 'Return code': 0, 'Message': None, 'User time': 0.027000205591321, 'System time': None, 'Wallclock time': None, 'Termination condition': 'optimal', 'Termination message': None}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [68]:
m2.A.pprint(), m2.B.pprint()

A : Technology Cost
    Size=3, Index=t
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
      R1 :     0 : 187.49999988936673 :  None : False : False : NonNegativeReals
      R2 :     0 :      3.6725835e-08 :  None : False : False : NonNegativeReals
    SEP1 :     0 : 249.99999992392503 :  None : False : False : NonNegativeReals
B : Material Cost
    Size=4, Index=j
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      A :  None :   624.9999996286922 :  None : False : False :  Reals
      B :  None :      2.15217179e-07 :  None : False : False :  Reals
      C :  None : -349.99999996782833 :  None : False : False :  Reals
      D :  None :  -599.9999997296313 :  None : False : False :  Reals


(None, None)

In [69]:
m2.F[1,"A"].pprint(), m2.F[2,"B"].pprint(), m2.F[5,"C"].pprint(), m2.F[6,"D"].pprint()

{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value              : Upper : Fixed : Stale : Domain
    (1, 'A') :     0 : 124.99999992573844 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value        : Upper : Fixed : Stale : Domain
    (2, 'B') :     0 : 3.586953e-08 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value              : Upper : Fixed : Stale : Domain
    (5, 'C') :     0 : 49.999999995404046 :  None : False : False : NonNegativeReals
{Member of F} : Flow rate of stream i speices j
    Size=24, Index=F_index
    Key      : Lower : Value             : Upper : Fixed : Stale : Domain
    (6, 'D') :     0 : 74.99999996620392 :  None : False : False : NonNegativeReals


(None, None, None, None)

In [70]:
m2.eps.pprint()

eps : uncertainty(Phi: adjustable parameter)
    Size=2, Index=jfeed
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      A :  -0.2 : -0.004247842147799 :   0.2 : False : False :  Reals
      B :  -0.1 : -0.001082959026694 :   0.1 : False : False :  Reals


### Now we consider a case with coefficient uncertainty

A very small example:

$$ A \longrightarrow C \quad \text{RXN1}$$
$$ B \longrightarrow C \quad \quad \text{RXN2}$$


